In [1]:
#code for STA/LTA detections 
using Plots
using Statistics

#sample data object
struct sampleData
    trace
    sampleRate
    startTime
end

#make sample data - random signal centered on 0
sample = sampleData(rand(10000).- 0.5,10,0)

#add an impulsive spike
sample.trace[4000] = 2.5
sample.trace[4002] = 2.5
sample.trace[4004] = 2.5
sample.trace[4006] = 2.5
sample.trace[4008] = 2.5
sample.trace[4010] = 2.5
sample.trace[4012] = 2.5
sample.trace[4014] = 2.5
sample.trace[4016] = 2.5
sample.trace[4018] = 2.5

sample.trace[4001] = -2.5
sample.trace[4003] = -2.5
sample.trace[4005] = -2.5
sample.trace[4007] = -2.5
sample.trace[4009] = -2.5
sample.trace[4011] = -2.5
sample.trace[4013] = -2.5
sample.trace[4015] = -2.5
sample.trace[4017] = -2.5
sample.trace[4019] = -2.5


function detect(data,longWinLength::Int=20,shortWinLength::Int=2,threshold::Int=2,overlap::Int=longWinLength/2)
    #convert window lengths from seconds to samples
    longWinLength = longWinLength * data.sampleRate
    shortWinLength = shortWinLength * data.sampleRate

    numLongWin = length(data.trace) / longWinLength
    numShortWin = longWinLength / shortWinLength
    detections = Float32[]

    #loop through data in long windows
    for i = 1:Int(numLongWin)
        startSampleLong = (i-1) * longWinLength + 1
        finishSampleLong = i * longWinLength
        longTrace = data.trace[startSampleLong:finishSampleLong]
        lta = mean( broadcast(abs,longTrace))

        #loop through long window in short windows  
        for n = 1:Int(numShortWin)
            startSampleShort = startSampleLong + (n-1) * shortWinLength + 1
            finishSampleShort = startSampleLong + (n) * shortWinLength - 1
            shortTrace = data.trace[startSampleShort:finishSampleShort]
            sta = mean(broadcast(abs,shortTrace))
            staLta = sta / lta

            #record detection time if trigger condition is met
            if staLta > threshold
                trigger = data.startTime + (startSampleShort / data.sampleRate)
                detections = [detections;trigger]
            end
        end
    end  
    
    return detections
    
end

detections = detect(sample,50)
print(detections)
plot(sample.trace)


ArgumentError: ArgumentError: Package Plots not found in current path:
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.
